In [1]:
from collections import defaultdict
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms
import re


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024*5)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def create_model(X, Y):
    model = Sequential(
        [
            LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            Dropout(0.25),
            LSTM(256),
            Dense(Y.shape[1], activation="softmax")
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer="adam")
    return model

In [5]:
def text(filename):
    text_load = re.sub(r"[ё]", "е", open(filename, encoding="utf8").read().lower())
    text_load = re.sub(r"[\u00A0\u00A0\u00A0\\n\\r\"\\«»#'<>&$()*-0123456789:;\[\]_`abcdefghijklmnopqrstuvwxyz]", " ", text_load)
    text_load = re.sub(r"[^а-я\s.]", ".", text_load)
    text_load = re.sub(r"\s+", " ", text_load)
    letters = sorted(list(set(text_load)))
    table = {}
    index = {}
    for i, letter in enumerate(letters):
        table[letter] = i
        index[i] = letter
    print(table)
    print(index)
    print(text_load[:300])
    return text_load, table, index, letters

In [ ]:
text_load, table, ind, letters = text("./data/Приключения Тома Сойера.txt")

In [ ]:
length = 10
dataX = []
dataY = []
sentences = text_load.split(".")
for sentence in sentences:
    sentence += "."
    for i in range(len(sentence) - length):
        dataX.append(sentence[i:i + length])
        dataY.append(sentence[i + length])


In [8]:
len(sentences), len(dataX)

(1202, 337514)

In [ ]:
X, Y = np.zeros(((len(dataX)), length, len(table))), np.zeros(((len(dataX)), len(table)))

for i, sentence in enumerate(dataX):
    for j, letter in enumerate(sentence):
        X[i, j, table[letter]] = 1
    Y[i, table[dataY[i]]] = 1


In [10]:
X.shape, Y.shape

((337514, 10, 34), (337514, 34))

In [11]:
model = create_model(X, Y)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 512)           1120256   
                                                                 
 dropout (Dropout)           (None, 10, 512)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 34)                8738      
                                                                 
Total params: 1,916,450
Trainable params: 1,916,450
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X, Y, epochs=10)

Epoch 1/10
10548/10548 [==============================] - 59s 5ms/step - loss: 2.0043
Epoch 2/10
10548/10548 [==============================] - 54s 5ms/step - loss: 1.6185
Epoch 3/10
10548/10548 [==============================] - 58s 5ms/step - loss: 1.4969
Epoch 4/10
10548/10548 [==============================] - 54s 5ms/step - loss: 1.4188
Epoch 5/10
10548/10548 [==============================] - 75s 7ms/step - loss: 1.3584
Epoch 6/10
10548/10548 [==============================] - 107s 10ms/step - loss: 1.3079
Epoch 7/10
10548/10548 [==============================] - 105s 10ms/step - loss: 1.2631
Epoch 8/10
10548/10548 [==============================] - 100s 9ms/step - loss: 1.2215
Epoch 9/10
10548/10548 [==============================] - 100s 10ms/step - loss: 1.1813
Epoch 10/10
10548/10548 [==============================] - 102s 10ms/step - loss: 1.1493


In [13]:
lstm_seq = 10
iterations = 30

In [14]:
def lstm_predict(inp):
    inp = inp.lower()[:lstm_seq]
    print(f"Sample: \t `{inp}`")
    predict = inp
    for _ in range(iterations):
        x = np.zeros((1, length, len(table)))
        for i, letter in enumerate(inp):
            x[0, i, table[letter]] = 1
        next = ind[np.argmax(model.predict(x, verbose=0))]
        inp = inp.pop(0) + next
        predict += next
    print(f"Predict:\t `{predict}`")
    print()

In [15]:
for _ in range(10):
    lstm_predict(dataX[np.random.randint(len(dataX) - 1)])


Sample: 	 `дя на свое`
Predict:	 `дя на своем долго словно остальные событ`

Sample: 	 `арыты в ка`
Predict:	 `арыты в каком нибудь под конец все таки `

Sample: 	 `ималось пр`
Predict:	 `ималось просто на свете он поднимал как `

Sample: 	 `ошачьи они`
Predict:	 `ошачьи они не собралась в полночь он под`

Sample: 	 `когда не п`
Predict:	 `когда не подумал а то что то не слышал к`

Sample: 	 `взглянул н`
Predict:	 `взглянул на него скоро он сказал том отд`

Sample: 	 `е было вос`
Predict:	 `е было воскресенье он не слышал как он н`

Sample: 	 `мэррела ск`
Predict:	 `мэррела сколько просто не было не подума`

Sample: 	 `сведи мне `
Predict:	 `сведи мне не сказал том отделил его до т`

Sample: 	 `а что если`
Predict:	 `а что если бы он не мог придумать свой д`



In [16]:
lstm_predict("Пожмите мне руку - ваши-то пролезут сквозь решетку, а моя нет, слишком велика.")

Sample: 	 `пожмите мн`
Predict:	 `пожмите мне подумал а то что то не слыша`



In [17]:
lstm_predict("Дальше в переулке он застал Джо Гарпера с Геком Финном")

Sample: 	 `дальше в п`
Predict:	 `дальше в полночь он поднимал как то в го`



In [18]:
lstm_predict("Шайка Тома Сойера здорово получается, правда")

Sample: 	 `шайка тома`
Predict:	 `шайка тома не было не подумал а то что т`



In [19]:
lstm_predict("Мальчики после этого играли в цирк целых")

Sample: 	 `мальчики п`
Predict:	 `мальчики подождали кошкой и положив под `



In [20]:
lstm_predict("Джим Холлис, которого он долго разыскивал, сказал,")

Sample: 	 `джим холли`
Predict:	 `джим холлис выследить на колени и стали `



In [21]:
class MarkovChain:
    def __init__(self, sentences, n):
        self.n = n
        self.dataX = []
        letters = []
        for sentence in sentences:
            sentence += "."
            for i in range(len(sentence) - n):
                self.dataX.append(sentence[i : i + n])
                letters.append(sentence[i + n])
        self.edges = defaultdict(lambda: defaultdict(lambda: 0))
        for sentence, letter in zip(self.dataX, letters):
            self.edges[sentence][letter] += 1

    def predict(self, string):
        string = string.lower()[:self.n]
        ans = string
        print(ans)
        for _ in range(50):
            next = [letter for letter in self.edges[string]]
            predict = np.array([_ for _ in self.edges[string].values()])
            if len(predict) == 0:
                break
            predict = predict / predict.sum()
            letter = next[np.argmax(predict)]
            string = string[1:] + letter
            ans += letter
        return ans

In [22]:
seq = 8
markov = MarkovChain(sentences, seq)

In [23]:
markov.predict("Пожмите мне руку - ваши-то пролезут сквозь решетку, а моя нет, слишком велика.")

пожмите 


'пожмите мне в глава проснувшихся сторожевых собак у них на'

In [24]:
markov.predict("Дальше в переулке он застал Джо Гарпера с Геком Финном")

дальше в


'дальше вздрагивая каждый день а вот джеф тэтчер побледнела'

In [25]:
markov.predict("Шайка Тома Сойера здорово получается, правда".lower()[:seq])

шайка то


'шайка тома сказала что он сам так безбоязненно идет к ней '

In [26]:
markov.predict("Мальчики после этого играли в цирк целых")

мальчики


'мальчики все теперь он покраснела и вообще не очень огорчи'

In [27]:
markov.predict("Джим Холлис, которого он долго разыскивал, сказал,")

джим хол


'джим холлис которой он обращался домой в самом деле то ест'